# Example stitching workflow

Using a small example dataset with two positions.

In [3]:
VISUALIZE_USING_NAPARI = False

In [1]:
import os, tempfile
import numpy as np

from multiview_stitcher import sample_data, msi_utils

import dask.diagnostics

2023-09-24 14:23:08,354 [Geometry3D WARNING] 'Renderer' is deprecated, using 'Visualizer' instead.


In [4]:
if VISUALIZE_USING_NAPARI:
    from napari_stitcher import viewer_utils
    import napari
    viewer = napari.Viewer()

In [5]:
# Load dataset from file

from multiview_stitcher import io

filename = sample_data.get_mosaic_sample_data_path() # or 'my_file.czi'
sims = io.read_mosaic_image_into_list_of_spatial_xarrays(filename)
msims = [msi_utils.get_msim_from_sim(sim, scale_factors=[]) for sim in sims]
print(f"Loaded {len(msims)} positions from {filename}. Transform key is {io.METADATA_TRANSFORM_KEY}.")

# Test with generated dataset

# sims = sample_data.generate_tiled_dataset(
#     ndim=2,
#     N_t=1,
#     N_c=1,
#     tile_size=1000,
#     tiles_x=5,
#     tiles_y=5,
#     zoom=4,
#     overlap=30,
#     drift_scale=0,
#     shift_scale=15,
# )

Loaded 2 positions from /Users/malbert/software/multiview-stitcher/image-datasets/mosaic_test.czi. Transform key is affine_metadata.


In [6]:
if VISUALIZE_USING_NAPARI:
    
    layer_tuples = viewer_utils.create_image_layer_tuples_from_msims(
        msims, n_colors=3, transform_key='affine_metadata')
    viewer_utils.add_image_layer_tuples_to_viewer(viewer, layer_tuples)
    viewer.reset_view()

In [4]:
from multiview_stitcher import registration, fusion

import importlib
registration = importlib.reload(registration)

print('Registering...')
with dask.diagnostics.ProgressBar():
    params = registration.register(
        msims, transform_key='affine_metadata', reg_channel_index=0)
    
for msim, param in zip(msims, params):
    msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_metadata')

print('Fusing...')
with dask.diagnostics.ProgressBar():
    fused = fusion.fuse(
        [msi_utils.get_sim_from_msim(msim) for msim in msims],
        transform_key='affine_registered')

mfused = msi_utils.get_msim_from_sim(fused, scale_factors=[])

Registering...
[######################################  ] | 95% Completed | 334.24 mshello5
hello5
hello5
hello5
[########################################] | 100% Completed | 439.69 ms
Fusing...


In [5]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=2)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_registered', n_colors=4,
        name_prefix='pre-registered view')
    mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msim(
        mfused, transform_key='affine_registered',
        name_prefix='registered view')
    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

In [6]:
with dask.diagnostics.ProgressBar():
    io.save_sim_as_tif('fused.tif', fused)

[########################################] | 100% Completed | 212.90 ms
